В рамках задания **Ультра-ПРО** вам предлагается воспользоваться фреймворком **Chainer**.

Используя произвольный датасет (CIFAR10, CIFAR100 или любой другой, в том числе свой собственный), создайте и обучите GAN с применением инструментов фреймворка **Chainer**

[Ссылка на документацию](https://docs.chainer.org/en/stable/index.tml)

Установка chainer:

In [ ]:
!pip install chainer

     |████████████████████████████████| 1.0 MB 8.8 MB/s 
  Created wheel for chainer: filename=chainer-7.8.0-py3-none-any.whl size=967734 sha256=f9ae18dacfe1b53b6c42982b3c769ae2456908859f2afbd39f3fd61a1b0fd4d2
  Stored in directory: /root/.cache/pip/wheels/8d/45/4c/c961846626d4db2511e4acbd5056eefb2aeedcec8208a3ca42
Successfully built chainer


In [ ]:
# Импорт библиотеки chainer
import chainer

# Присвоим псевдоним ее функциям
import chainer.functions as F

# Присвоим псевдоним ее ссылкам
import chainer.links as L

# Загрузим библиотеку chainerx
import chainerx

# Загрузим функции работы с GPU
import chainer.backends.cuda

# Загрузим массив со структурой для отслеживания вычислений
from chainer import Variable

# Загрузим стандартный обучающий массив из chainer
from chainer import training

# Импорт расширений
from chainer.training import extensions

# Библиотека для работы с файловой системой виртуальной машины
import os

# Numpy массивы
import numpy as np

# Модуль для обработки аргументов командной строки
import argparse

# Модуль предупреждений
import warnings

# # Библиотека для отрисовки графиков
import matplotlib.pyplot as plt

# Модуль отрисовки изображений
from PIL import Image

/usr/local/lib/python3.7/dist-packages/chainer/_environment_check.py:75: UserWarning: 
--------------------------------------------------------------------------------
CuPy (cupy-cuda111) version 9.4.0 may not be compatible with this version of Chainer.
Please consider installing the supported version by running:
  $ pip install 'cupy-cuda111>=7.7.0,<8.0.0'

See the following page for more details:
  https://docs.cupy.dev/en/latest/install.html
--------------------------------------------------------------------------------

  requirement=requirement, help=help))


## Классы 

### Класс DCGANUpdate

Он будет отвечать за вычисление ошибок генератора и дискриминатора, а также за их оптимизацию.




In [ ]:
class DCGANUpdater(chainer.training.updaters.StandardUpdater):
    '''
    Данный класс является наследником класса chainer.training.updaters.StandardUpdater.
    При инициализации создает объект из двух классов - генератора и дискриминатора.

    Содержит функции: 
    loss_dis - ошибка дискриминатора;
    loss_gen - ошибка генератора;
    update_core - дообучает генератор и дискриминатор
    '''

    def __init__(self, 
                 *args, 
                 **kwargs):                                        # Конструктор класса
        
        self.gen, self.dis = kwargs.pop('models')                               # Создаем генератор и дискриминатор  
        super(DCGANUpdater, self).__init__(*args, **kwargs)                     # Получим доступ к методам класса родителя

    def loss_dis(self, 
                 dis, 
                 y_fake, 
                 y_real):                                    # Создадим функцию подсчета ошибки дискриминатора
        '''
        Данная функция занимается подсчетом ошибки дискриминатора

        Args:
          self - аргумент ссылка на атрибуты
          dis - модель дискриминатора
          y_fake - ложная картинка
          y_real - реальная картинка

        Returns:
          loss - ошибка дискриминатора
        '''

        batchsize = len(y_fake)                                                 # Длина массива ложной картинки
        L1 = F.sum(F.softplus(-y_real)) / batchsize                             # Вычислим ошибку на реальной картинке  
        L2 = F.sum(F.softplus(y_fake)) / batchsize                              # Вычислим ошибку на ложной картинке
        loss = L1 + L2                                                          # Суммируем обе ошибки
        chainer.report({'loss': loss}, dis)                                     # Сообщает наблюдаемые значения 

        # Возвращаем ошибку дискриминатора
        return loss                                           

    def loss_gen(self, 
                 gen, 
                 y_fake):                                            # Создадим функцию подсчета ошибки генератора
        '''
        Данная функция занимается подсчетом ошибки генератора

        Args:
          self - аргумент ссылка на атрибуты
          gen - модель генератора
          y_fake - ложная картинка

        Returns:
          loss - ошибка дискриминатора
        '''

        batchsize = len(y_fake)                                                 # Длина массива ложной картинки
        loss = F.sum(F.softplus(-y_fake)) / batchsize                           # Вычислим ошибку на ложной картинке
        chainer.report({'loss': loss}, gen)                                     # Сообщает наблюдаемые значения

        # Возвращаем ошибку генератора
        return loss

    def update_core(self):                                                      # Создадим функцию изменения параметров ген. и диск.
        '''
        Данная функция занимается обучением генеартора и дискриминатора по полученным данным от
        функций loss_dis и loss_gen

        Args:
          self - аргумент ссылка на атрибуты

        Функция ничего не возвращает
        '''

        gen_optimizer = self.get_optimizer('gen')                               # Получение данных оптимизатора для генератора    
        dis_optimizer = self.get_optimizer('dis')                               # Получение данных оптимизатора для дискриминатора

        batch = self.get_iterator('main').next()                                # В батч запишем очередную итерацию
        device = self.device                                                    # Возьмем параметры оборудования  
        x_real = Variable(self.converter(batch, device)) / 255.                 # Конвертируем и нормализуем данные

        gen, dis = self.gen, self.dis                                           # Запишем атрибуты моделей
        batchsize = len(batch)                                                  # Длина массива батча

        y_real = dis(x_real)                                                    # В дискриминатор запишем действительную картинку

        z = Variable(device.xp.asarray(gen.make_hidden(batchsize)))             # Подготовим картинку для подачи в генератор
        x_fake = gen(z)                                                         # Получаем картинку из генератора
        y_fake = dis(x_fake)                                                    # Теперь получим ложную картинку из дискриминатора    

        dis_optimizer.update(self.loss_dis, dis, y_fake, y_real)                # Обновим параметры оптимизатора из дискриминатора
        gen_optimizer.update(self.loss_gen, gen, y_fake)                        # Обновим параметры оптимизатора из генератора

### Генератор

Поскольку мы решили использовать ООП в нашем проекте, то надо сделать отдельные классы для генератора и дискриминатора. 

Начнем с создания класса генератора (Generator). Архитектура, которую предлагает нам библиотека chainer довольно специфичная и не похожа на ту что мы привыкли в keras. Тут создается отдельная функция для слоев, отдельная функция для активационных функций. 

In [ ]:
class Generator(chainer.Chain):
    '''
    Данный класс отвечает за создание объекта Генератор. Позволяет собрать архитектуру нейронной сети.
    Является дочерним от класса chainer.Chain

    Содержит функции:
    make_hidden - сделает массив для случайной картинки
    forward     - набор активационных функций
    '''  
    def __init__(self, 
                 n_hidden, 
                 bottom_width=4, 
                 ch=512, 
                 wscale=0.02):          # Конструктор класса

        super(Generator, self).__init__()                                       # Создаем объект
        self.n_hidden = n_hidden                                                # Зададим атрибут n_hidden
        self.ch = ch                                                            # Зададим атрибут ch
        self.bottom_width = bottom_width                                        # Зададим атрибут bottom_width (габарит картинки)

        with self.init_scope():                                                 # Начинаем сборку сети
            w = chainer.initializers.Normal(wscale)                             # Инициализирует массив с нормальным распределением
            self.l0 = L.Linear(self.n_hidden, bottom_width * bottom_width * ch, # Линейный слой (он же полносвязный)
                               initialW=w)
            self.dc1 = L.Deconvolution2D(ch, ch // 2, 4, 2, 1, initialW=w)      # Добавим слой развертки 
            self.dc2 = L.Deconvolution2D(ch // 2, ch // 4, 4, 2, 1, initialW=w) # Добавим слой развертки 
            self.dc3 = L.Deconvolution2D(ch // 4, ch // 8, 4, 2, 1, initialW=w) # Добавим слой развертки  
            self.dc4 = L.Deconvolution2D(ch // 8, 3, 3, 1, 1, initialW=w)       # Добавим слой развертки 
            self.bn0 = L.BatchNormalization(bottom_width * bottom_width * ch)   # Добавим слой нормализации
            self.bn1 = L.BatchNormalization(ch // 2)                            # Добавим слой нормализации 
            self.bn2 = L.BatchNormalization(ch // 4)                            # Добавим слой нормализации
            self.bn3 = L.BatchNormalization(ch // 8)                            # Добавим слой нормализации 

    def make_hidden(self, 
                    batchsize):               
        '''
        Функция генерирует случайный массив (картинку) для подачи в генератор 

        Args:
          self      - аргумент ссылка на атрибуты
          batchsize - размер батча
        
        Returns:
          Массив случайных значений размером (батчсайз,n_hidden,1,1)  
        '''
        dtype = chainer.get_dtype()                                             # Получим тип создаваемого объекта

        # Вернем Numpy-массив случайных значений от -1 до 1 нужного размера 
        return np.random.uniform(-1, 1, (batchsize, self.n_hidden, 1, 1))\
            .astype(dtype)

    def forward(self, 
                z):
        '''
        Функция предоставляет набор активационных функций

        Args:
          self - аргумент ссылка на атрибуты
          z    - аргумент для подачи картинки
        Returns:
          x    - массив, который преобразовывается в картинку
        '''
        h = F.reshape(F.relu(self.bn0(self.l0(z))),                             # Активационная функция relu
                      (len(z), self.ch, self.bottom_width, self.bottom_width))  # Здесь она также преобразовывается 
        h = F.relu(self.bn1(self.dc1(h)))                                       # Добавим еще одну relu-функцию  
        h = F.relu(self.bn2(self.dc2(h)))                                       # Добавим еще одну relu-функцию
        h = F.relu(self.bn3(self.dc3(h)))                                       # Добавим еще одну relu-функцию
        x = F.sigmoid(self.dc4(h))                                              # Добавим sigmoid-функцию (на самом последнем слое)

        # Вернем что получилось в итоге
        return x

### Класс Дискриминатор

Теперь создадим класс для дискриминатора (Discriminator). 

In [ ]:
class Discriminator(chainer.Chain):
    '''
    Данный класс отвечает за создание объекта Дискриминатор. Позволяет собрать архитектуру нейронной сети.
    Является дочерним от класса chainer.Chain

    Содержит функцию:
    forward     - набор активационных функций
    '''  
    def __init__(self, 
                 bottom_width=4, 
                 ch=512, 
                 wscale=0.02):                    # Конструктор класса 
                 
        w = chainer.initializers.Normal(wscale)                                 # Инициализирует массив с нормальным распределением
        super(Discriminator, self).__init__()                                   # Создаем объект  
        with self.init_scope():
            self.c0_0 = L.Convolution2D(3, ch // 8, 3, 1, 1, initialW=w)        # Добавим сверточный слой
            self.c0_1 = L.Convolution2D(ch // 8, ch // 4, 4, 2, 1, initialW=w)  # Добавим сверточный слой   
            self.c1_0 = L.Convolution2D(ch // 4, ch // 4, 3, 1, 1, initialW=w)  # Добавим сверточный слой
            self.c1_1 = L.Convolution2D(ch // 4, ch // 2, 4, 2, 1, initialW=w)  # Добавим сверточный слой
            self.c2_0 = L.Convolution2D(ch // 2, ch // 2, 3, 1, 1, initialW=w)  # Добавим сверточный слой
            self.c2_1 = L.Convolution2D(ch // 2, ch // 1, 4, 2, 1, initialW=w)  # Добавим сверточный слой
            self.c3_0 = L.Convolution2D(ch // 1, ch // 1, 3, 1, 1, initialW=w)  # Добавим сверточный слой
            self.l4 = L.Linear(bottom_width * bottom_width * ch, 1, initialW=w) # Добавим полносвязный слой (l4 - выходной)
            self.bn0_1 = L.BatchNormalization(ch // 4, use_gamma=False)         # Добавим слой нормализации
            self.bn1_0 = L.BatchNormalization(ch // 4, use_gamma=False)         # Добавим слой нормализации
            self.bn1_1 = L.BatchNormalization(ch // 2, use_gamma=False)         # Добавим слой нормализации
            self.bn2_0 = L.BatchNormalization(ch // 2, use_gamma=False)         # Добавим слой нормализации
            self.bn2_1 = L.BatchNormalization(ch // 1, use_gamma=False)         # Добавим слой нормализации
            self.bn3_0 = L.BatchNormalization(ch // 1, use_gamma=False)         # Добавим слой нормализации

    def forward(self, x):
        '''
        Функция предоставляет набор активационных функций

        Args:
          self - аргумент ссылка на атрибуты
          x    - аргумент для подачи картинки из генератора
        Returns:
          self.l4(h) - в данном случае выдает результат работы полносвязного 
          слоя размерностью (bottom_width * bottom_width * ch)
        '''
        device = self.device                                                    # Параметры оборудования
        h = add_noise(device, x)                                                # Добавим шум
        h = F.leaky_relu(add_noise(device, self.c0_0(h)))                       # Добавим активационную функцию LeakyReLU 
        h = F.leaky_relu(add_noise(device, self.bn0_1(self.c0_1(h))))           # Добавим активационную функцию LeakyReLU
        h = F.leaky_relu(add_noise(device, self.bn1_0(self.c1_0(h))))           # Добавим активационную функцию LeakyReLU
        h = F.leaky_relu(add_noise(device, self.bn1_1(self.c1_1(h))))           # Добавим активационную функцию LeakyReLU  
        h = F.leaky_relu(add_noise(device, self.bn2_0(self.c2_0(h))))           # Добавим активационную функцию LeakyReLU
        h = F.leaky_relu(add_noise(device, self.bn2_1(self.c2_1(h))))           # Добавим активационную функцию LeakyReLU
        h = F.leaky_relu(add_noise(device, self.bn3_0(self.c3_0(h))))           # Добавим активационную функцию LeakyReLU

        # Вернем результат работы полносвязного слоя дискриминатора
        return self.l4(h)

## Функции

### Вывод картинок

In [ ]:
def out_generated_image(gen, 
                        dis, 
                        rows, 
                        cols, 
                        seed, 
                        dst):
    '''
    Данная функция занимается созданием картинки для сохранения ее на виртуальной машине

    Args:
      gen  - модель генератора
      dis  - модель дискриминатора (внутри функции не используется)
      rows - количество строк
      cols - количество столбцов
      seed - начальные условия для генератора случайных чисел
      dst  - адрес папки проекта

    Returns:
      make_image - результат работы внутреней функции 
    '''
    @chainer.training.make_extension()                                          # Декоратор для превращения заданных функций 
    def make_image(trainer):                                                    
        '''
        Данная функция занимается преобразованием полученных данных из 
        генератора в набор картинок
        
        Args:
          trainer - объект для обучения сети

        Функция ничего не возвращает 
        '''
        np.random.seed(seed)                                                    # "Останавливает" генератор случайных чисел  
        n_images = rows * cols                                                  # Задаем количество картинок
        xp = gen.xp                                                             # Параметры оборудования от генератора
        z = Variable(xp.asarray(gen.make_hidden(n_images)))                     # Создадим массив картинок  
        with chainer.using_config('train', False):                              # При этом используя режим обучения 'train'  
            x = gen(z)                                                          # для генератора
        x = chainer.backends.cuda.to_cpu(x.array)                               # Копирует выданный GPU массив в CPU 
        np.random.seed()                                                        # Снова задаем генератор случайных чисел

        x = np.asarray(np.clip(x * 255, 0.0, 255.0), dtype=np.uint8)            # Создадим целочисленный массив   
        _, _, H, W = x.shape                                                    # Зададим высоту (H) и ширину (W) одной картинки  
        x = x.reshape((rows, cols, 3, H, W))                                    # Решэйпим картинки
        x = x.transpose(0, 3, 1, 4, 2)                                          # Транспонируем полученный массив
        x = x.reshape((rows * H, cols * W, 3))                                  # Создаем цветной массив из картинок (rows,cols)  

        preview_dir = '{}/preview'.format(dst)                                  # Сформируем адрес папки для preview
        # Сформируем адрес папки image и сохраним картинку
        preview_path = preview_dir +'/image{:0>8}.png'.format(trainer.updater.iteration) 
                            
             
        # Если папка preview_dir не существует, 
        if not os.path.exists(preview_dir):
            # нужно создать эту папку
            os.makedirs(preview_dir)
        
        # Получим картинку из массива x и сохраним в папку preview_path
        Image.fromarray(x).save(preview_path)

    # Вернем результат работы функции make_image    
    return make_image

### Добавление шума

In [ ]:
def add_noise(device, 
              h, 
              sigma=0.2):
    '''
    Функция добавления шума.

    Args:
      device - параметры оборудования
      h - параметры слоя
    
    Returns:
      h - параметры слоя     
    '''

    # Если включен режим обучения ('train')
    if chainer.config.train:
        # Параметры оборудования
        xp = device.xp

        # Если описание оборудования поддерживается chainerx
        if device.xp is chainerx:
            # Определим резервное устройство (для устройства native: 0 это CpuDevice, для cuda: 1 - GpuDevice)
            # Подробнее об атрибуте см. https://docs.chainer.org/en/latest/reference/generated/chainer.backend.ChainerxDevice.html
            fallback_device = device.fallback_device
            # Используя данные параметры
            with chainer.using_device(fallback_device):
                # передаем заданные массивы на устройство
                randn = device.send(fallback_device.xp.random.randn(*h.shape))
        else:
            # В противном случае просто заполним случайными данными существующий слой
            randn = xp.random.randn(*h.shape)

        # Вернем данные слоя плюс случайные шумы    
        return h + sigma * randn
    # Если режим обучения ('train') не включен,
    else:
        # то возвращаем только данные слоя
        return h

### Запуск обучения

Для начала разберем модуль работы с командной строкой (argparse). Есть у него функция ArgumentParser, которая позволяет собрать набор аргументов.   

## Строка аргументов

In [ ]:
# Инициализируем создание объекта парсера аргументов
 
parser = argparse.ArgumentParser(description='Chainer example: DCGAN')

In [ ]:
# Введем аргумент отвечающий за размер батча (batchsize). Определим его тип и размер

parser.add_argument('--batchsize', '-b', type=int, default=50,help='Number of images in each mini-batch')

_StoreAction(option_strings=['--batchsize', '-b'], dest='batchsize', nargs=None, const=None, default=50, type=<class 'int'>, choices=None, help='Number of images in each mini-batch', metavar=None)

In [ ]:
# Введем аргумент отвечающий за количество эпох (epoch). Определим тип и количество

parser.add_argument('--epoch', '-e', type=int, default=2,help='Number of sweeps over the dataset to train')

_StoreAction(option_strings=['--epoch', '-e'], dest='epoch', nargs=None, const=None, default=2, type=<class 'int'>, choices=None, help='Number of sweeps over the dataset to train', metavar=None)

In [ ]:
# Введем аргумент отвечающий за оборудование. Спецификатор устройства. Либо спецификатор устройства ChainerX, либо целое число. 
# Если целое неотрицательное число, Используются массивы CuPy с указанным идентификатором устройства. 
# Если целое отрицательное, используются массивы NumPy

parser.add_argument('--device', '-d', type=str, default='-1',
                        help='Device specifier. Either ChainerX device '
                        'specifier or an integer. If non-negative integer, '
                        'CuPy arrays with specified device id are used. If '
                        'negative integer, NumPy arrays are used')

_StoreAction(option_strings=['--device', '-d'], dest='device', nargs=None, const=None, default='-1', type=<class 'str'>, choices=None, help='Device specifier. Either ChainerX device specifier or an integer. If non-negative integer, CuPy arrays with specified device id are used. If negative integer, NumPy arrays are used', metavar=None)

In [ ]:
# Введем аргумент отвечающий за датсет. Директорию с .img файлами. По умолчанию CIFAR10

parser.add_argument('--dataset', '-i', default='',help='Directory of image files.  Default is cifar-10.')

_StoreAction(option_strings=['--dataset', '-i'], dest='dataset', nargs=None, const=None, default='', type=None, choices=None, help='Directory of image files.  Default is cifar-10.', metavar=None)

In [ ]:
# Введем аргумент отвечающий за вывод результата. Название папки с полученными картинками

parser.add_argument('--out', '-o', default='result',help='Directory to output the result')

_StoreAction(option_strings=['--out', '-o'], dest='out', nargs=None, const=None, default='result', type=None, choices=None, help='Directory to output the result', metavar=None)

In [ ]:
# Введем аргумент для возобновления обучения

parser.add_argument('--resume', '-r', type=str,help='Resume the training from snapshot')

_StoreAction(option_strings=['--resume', '-r'], dest='resume', nargs=None, const=None, default=None, type=<class 'str'>, choices=None, help='Resume the training from snapshot', metavar=None)

In [ ]:
# Введем аргумент для количества скрытых элементов (z). Тип - int. По умолчанию - 100

parser.add_argument('--n_hidden', '-n', type=int, default=100,help='Number of hidden units (z)')

_StoreAction(option_strings=['--n_hidden', '-n'], dest='n_hidden', nargs=None, const=None, default=100, type=<class 'int'>, choices=None, help='Number of hidden units (z)', metavar=None)

In [ ]:
# Введем аргумент для случайного начального числа z на этапе визуализации. По умолчанию - 0

parser.add_argument('--seed', type=int, default=0,help='Random seed of z at visualization stage')

_StoreAction(option_strings=['--seed'], dest='seed', nargs=None, const=None, default=0, type=<class 'int'>, choices=None, help='Random seed of z at visualization stage', metavar=None)

In [ ]:
# Введем аргумент, который будет отвечать за интервал сохранения картинок. По умолчанию - 100

parser.add_argument('--snapshot_interval', type=int, default=100,help='Interval of snapshot')

_StoreAction(option_strings=['--snapshot_interval'], dest='snapshot_interval', nargs=None, const=None, default=100, type=<class 'int'>, choices=None, help='Interval of snapshot', metavar=None)

In [ ]:
# Введем аргумент, который будет отвечать за интервал отображения логов в консоли. По умолчанию - 100

parser.add_argument('--display_interval', type=int, default=100,help='Interval of displaying log to console')

_StoreAction(option_strings=['--display_interval'], dest='display_interval', nargs=None, const=None, default=100, type=<class 'int'>, choices=None, help='Interval of displaying log to console', metavar=None)

In [ ]:
# Сгруппируем аргументы в старшие аргументы

group = parser.add_argument_group('deprecated arguments')

In [ ]:
# Добавим аргумент, отвечающий за устройство на котором будет проводиться обучение

group.add_argument('--gpu', '-g', dest='device',type=int, nargs='?', const=0,help='GPU ID (negative value indicates CPU)')

_StoreAction(option_strings=['--gpu', '-g'], dest='device', nargs='?', const=0, default=None, type=<class 'int'>, choices=None, help='GPU ID (negative value indicates CPU)', metavar=None)

In [ ]:
# Создадим строку из наших аргументов

args, unknown = parser.parse_known_args()

In [ ]:
# Если тип данных с плавающей точкой

if chainer.get_dtype() == np.float16:
        # Отобразить предупреждение о то что пример может вызвать NaN в режиме FP16
        warnings.warn('This example may cause NaN in FP16 mode.', RuntimeWarning)

In [ ]:
# Возвращает объект устройства и включим

device = chainer.get_device(args.device)
device.use()

In [ ]:
# Выведем информацию

print('Device: {}'.format(device))                                              
print('# Minibatch-size: {}'.format(args.batchsize))
print('# n_hidden: {}'.format(args.n_hidden))
print('# epoch: {}'.format(args.epoch))
print('')

Device: @numpy
# Minibatch-size: 50
# n_hidden: 100
# epoch: 2



## Создаем объекты классов

In [ ]:
# Создадим объект Генератор

gen = Generator(n_hidden=args.n_hidden)

In [ ]:
# Создадим объект Дискриминатор

dis = Discriminator()

In [ ]:
# Разместим генератор на устройстве

gen.to_device(device) 

In [ ]:
# Разместим дискриминатор на устройстве

dis.to_device(device)

В библиотеке chainer есть свой оптимизатор Adam. Применим его. Подробнее про этот оптимизатор можно посмотреть здесь https://docs.chainer.org/en/latest/reference/generated/chainer.optimizers.Adam.html . 

In [ ]:
def make_optimizer(model, 
                   alpha=0.0002, 
                   beta1=0.5):
        '''
        Функция применяет оптимизатор к заданной модели.

        Args:
          model - модель для компиляции
          alpha - коэффициент обучения (learning_rate) 
          beta1 - коэффициент экспоненциальной скорости затухания момента первого порядка
        
        Returns:
          optimizer - готовая для обучения сеть
        '''
        optimizer = chainer.optimizers.legacy.Adam(alpha=alpha, beta1=beta1)           # Создадим объект оптимизатор
        optimizer.setup(model)                                                  # Применим к модели 
        optimizer.add_hook(                                                     # Функция перехвата
            chainer.optimizer_hooks.WeightDecay(0.0001), 'hook_dec')
        
        # Возвращает сокмпонованную сеть
        return optimizer

In [ ]:
# Оптимизировать генератор

opt_gen = make_optimizer(gen)

In [ ]:
# Оптимизировать дискриминатор

opt_dis = make_optimizer(dis)

In [ ]:
# Если аргумент датасета "пустой" (не используем свой датасет)
if args.dataset == '':
        # Загружаем CIFAR10 в обучающую выборку
        train, _ = chainer.datasets.get_cifar10(withlabel=False, scale=255.)

# В противном случае        
else:        
        all_files = os.listdir(args.dataset)                                    # Определим названия всех файлов из папки датасета
        image_files = [f for f in all_files if ('png' in f or 'jpg' in f)]      # Определим пути к файлам

        # Отобразим адрес папки и количество картинок которое она содержит
        print('{} contains {} image files'.format(args.dataset, len(image_files)))
        # Создадим обучающую выборку из нашего датасета 
        train = chainer.datasets.ImageDataset(paths=image_files, root=args.dataset) 

In [ ]:
# Установим итератор

train_iter = chainer.iterators.SerialIterator(train, args.batchsize)

In [ ]:
# Создадим объект класса DCGANUpdater (определили его выше)

updater = DCGANUpdater(models=(gen, dis),iterator=train_iter,optimizer={'gen': opt_gen, 'dis': opt_dis},device=device)

In [ ]:
# Установим объект класса Trainer 

trainer = training.Trainer(updater, (args.epoch, 'epoch'), out=args.out)

In [ ]:
# Выделим значения интервалов отображения картинок и логов в отдельные переменные 

snapshot_interval = (args.snapshot_interval, 'iteration')
display_interval = (args.display_interval, 'iteration')

In [ ]:
# Определим название файла для вывода логов после каждого интервала
trainer.extend(extensions.snapshot(filename='snapshot_iter_{.updater.iteration}.npz'),trigger=snapshot_interval)

In [ ]:
# Определим название файла для вывода логов генератора после каждого интервала

trainer.extend(extensions.snapshot_object(gen, 'gen_iter_{.updater.iteration}.npz'), trigger=snapshot_interval)

In [ ]:
# Определим название файла для вывода логов дискриминатора после каждого интервала

trainer.extend(extensions.snapshot_object(dis, 'dis_iter_{.updater.iteration}.npz'), trigger=snapshot_interval)

In [ ]:
# Расширение трейнера для вывода накопленных результатов в файл журнала
trainer.extend(extensions.LogReport(trigger=display_interval)) 

# Определим в каком виде будем отображать данные о прошедшей эпохе, итерации, ошибке генератора и дискриминатора
trainer.extend(extensions.PrintReport(['epoch', 'iteration', 'gen/loss', 'dis/loss',]), trigger=display_interval)

In [ ]:
# Отобразим прогресс бар

trainer.extend(extensions.ProgressBar(update_interval=10))

In [ ]:
# Укажем обучающему объекту функцию для отображения картинок

trainer.extend(out_generated_image(gen, dis,10, 10, args.seed, args.out),trigger=snapshot_interval)

In [ ]:
# Если аргументы не пустые
if args.resume is not None:
        # Загружаем объект из trainer в формате NPZ
        chainer.serializers.load_npz(args.resume, trainer)

In [ ]:
# Запускаем обучение

trainer.run()

     total [..................................................]  0.50%
this epoch [..................................................]  1.00%
        10 iter, 0 epoch / 2 epochs
       inf iters/sec. Estimated time to finish: 0:00:00.
     total [..................................................]  1.00%
this epoch [#.................................................]  2.00%
        20 iter, 0 epoch / 2 epochs
   0.11662 iters/sec. Estimated time to finish: 4:42:58.600409.
     total [..................................................]  1.50%
this epoch [#.................................................]  3.00%
        30 iter, 0 epoch / 2 epochs
   0.11913 iters/sec. Estimated time to finish: 4:35:36.375212.
     total [#.................................................]  2.00%
this epoch [##................................................]  4.00%
        40 iter, 0 epoch / 2 epochs
   0.12002 iters/sec. Estimated time to finish: 4:32:10.031155.
     total [#..........................

In [ ]:
# По данному адресу появится картинка

picture_path = '/content/result/preview/image00002000.png'

In [ ]:
# Загрузим картинку в переменную
image_1 = Image.open(picture_path) 

# Рисуем изображение
plt.imshow(image_1)
plt.show()